# CS6700: Reinforcement Learning
## Programming Assignment 3

Submitted by:
- Archish S (ME20B032)
- Vinayak Gupta (EE20B152)

# Semi-Markov Decision Process

## Imports

In [ ]:
import io
import glob
import random
from dataclasses import dataclass
from collections import deque
from itertools import count

import gym
from gym import wrappers
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"],
    "font.size": 10
})
%config InlineBackend.figure_format = 'retina'

import tqdm
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

## Environment

In [ ]:
env = gym.make('Taxi-v3')
state = env.reset()

# Current State
print("Current State:", env.s)

# Observation Space
locations = ["red", "green", "yellow", "blue"]
print ("Number of states:", env.observation_space.n)

# Action Space
actions = ["south", "north", "east", "west", "pick", "drop"]
print ("Number of actions:", env.action_space.n)

# Transition 
action = random.randint(0, len(actions)-1)
next_state, reward, done, info = env.step(action)
print("Action:", actions[action])
print("Next State:", next_state)
print("Reward:", reward)
print("Done:", done)

print("Decoded Current State:", list(env.decode(state)))
print("Decoded Next State:", list(env.decode(next_state)))

## Parameters

In [ ]:
max_runs = 5
max_episodes = 1000
max_steps = 500

epsilon_max = 0.99
epsilon_min = 0.01

gamma = 0.9
alpha = 0.1

nX = 5; nY = 5; nPas = 5; nDrop = 4
nS = nX * nY * nPas * nDrop

targets = {0: [0, 0], 1: [0, 4], 2: [4, 0], 3: [4, 3]}
nO = 4
nA = 6

assert nS == env.observation_space.n
assert nA == env.action_space.n

## Policy

In [ ]:
def egreedy_policy(q_values, state, epsilon):
    if q_values[state].any() and random.random() > epsilon:
        return np.argmax(q_values[state])

    choice = random.randint(0, q_values.shape[-1] - 1)  
    return choice

In [ ]:
def Option(env, state, Q, goal, epsilon=0.1):
    optdone = False
    x, y, pas, drop = env.decode(state)

    if (x == targets[goal][0] and y == targets[goal][1]):
        optdone = True 
        if pas == goal:
            optact = 4
        elif drop == goal:
            optact = 5 
        else:   
            optact = 1 if (goal in [0, 1]) else 0
    else:
        optact = egreedy_policy(Q, state, epsilon=epsilon) 
    return [optact, optdone]

# SMDP Q-Learning

In [ ]:
Q_SMDP = np.zeros((max_runs, nS, nA + nO))
Q_SMDP_Options = dict([(i, np.zeros((max_runs, nS, nA))) for i in range(nO)])

updates_SMDP = []

run_rewards = []
for run in range(max_runs):

    episode_updates = []
    episode_rewards = []
    for episode in range(max_episodes):
        state = env.reset()
        done = False
        
        step_updates = []
        step_rewards = 0
        step = 0
        while not done:
            epsilon = max(epsilon_min, epsilon_max - (epsilon_max - epsilon_min) * episode / max_episodes)

            x, y, pas, drop = env.decode(state)
            action = egreedy_policy(Q_SMDP[run], state, epsilon=epsilon)

            if action in [0, 1, 2, 3, 4, 5]:
                next_state, reward, done, info = env.step(action)

                Q_SMDP[run, state, action] = Q_SMDP[run, state, action] + alpha * (reward + gamma * np.max(Q_SMDP[run, next_state]) - Q_SMDP[run, state, action])
                
                step_rewards += reward
                step_updates.append(step)
                step += 1

                state = next_state

            else:
                start = state
                goal = action - 6

                optrewards = []
                optdone, optsteps = False, 0

                while not optdone:
                    optact, optdone = Option(env, state, Q_SMDP_Options[goal][run], goal, epsilon=epsilon)
                    next_state, reward, done, info = env.step(optact)

                    Q_SMDP_Options[goal][run, state, optact] = Q_SMDP_Options[goal][run, state, optact] + alpha * (reward + gamma * np.max(Q_SMDP_Options[goal][run, next_state]) - Q_SMDP_Options[goal][run, state, optact])

                    optrewards.append(gamma**optsteps * reward)
                    optsteps += 1
                    state = next_state

                optrewards = sum(optrewards)
                Q_SMDP[run, start, action] = Q_SMDP[run, start, action] + alpha * (optrewards - gamma**optsteps * np.max(Q_SMDP[run, next_state]) - Q_SMDP[run, start, action])

                step_rewards += optrewards
                step_updates.append(step + optsteps)
                step += optsteps


        episode_rewards.append(step_rewards)
        episode_updates.append(step_updates)

        clear_output(wait=True)
        print(f"Run: {run+1}/{max_runs}, Episode: {episode+1}/{max_episodes}: {step_rewards}")

    run_rewards.append(episode_rewards)
    updates_SMDP.append(episode_updates)

# Intra-Option Q-Learning

In [ ]:
Q_Intra = np.zeros((max_runs, nS, nA))
Q_Intra_Options = dict([(i, np.zeros((max_runs, nS, nA))) for i in range(nO)])

updates_Intra = []

run_rewards = []
for run in range(max_runs):

    episode_updates = []
    episode_rewards = []
    for episode in range(max_episodes):
        state = env.reset()
        done = False
        
        step_updates = []
        step_rewards = 0
        step = 0
        while not done:
            epsilon = max(epsilon_min, epsilon_max - (epsilon_max - epsilon_min) * episode / max_episodes)

            x, y, pas, drop = env.decode(state)
            action = egreedy_policy(Q_Intra[run], state, epsilon=epsilon)

            if action in [0, 1, 2, 3, 4, 5]:
                next_state, reward, done, info = env.step(action)

                Q_Intra[run, state, action] = Q_Intra[run, state, action] + alpha * (reward + gamma * np.max(Q_Intra[run, next_state]) - Q_Intra[run, state, action])
                
                step_rewards += reward
                step_updates.append(step)
                step += 1

                state = next_state

            else:
                start = state
                goal = action - 6

                optrewards = []
                optdone, optsteps = False, 0

                while not optdone:
                    optact, optdone = Option(env, state, Q_Intra_Options[goal][run], goal, epsilon=epsilon)
                    next_state, reward, done, info = env.step(optact)
                    next_x, next_y, next_pas, next_drop = env.decode(next_state)

                    Q_Intra_Options[goal][run, state, optact] = Q_Intra_Options[goal][run, state, optact] + alpha * (reward + gamma * np.max(Q_Intra_Options[goal][run, next_state]) - Q_Intra_Options[goal][run, state, optact])

                    for G in range(nO):
                        if np.argmax(Q_Intra_Options[G][run, next_state]) != optact and G != goal:
                            continue
                            
                        if next_x == targets[G][0] and next_y == targets[G][1]:
                            Q_Intra[run, state, G + 6] = Q_Intra[run, state, G + 6] + alpha * (reward + gamma * np.max(Q_Intra[run, next_state]) - Q_Intra[run, state, G + 6])
                        else:
                            Q_Intra[run, state, G + 6] = Q_Intra[run, state, G + 6] + alpha * (reward + gamma * Q_Intra[run, next_state, G + 6] - Q_Intra[run, state, G + 6])
                    
                    step_updates.append(step + optsteps)

                    optrewards.append(gamma**optsteps * reward)
                    optsteps += 1
                    state = next_state

                optrewards = sum(optrewards)
                step_rewards += optrewards
                step += optsteps


        episode_rewards.append(step_rewards)
        episode_updates.append(step_updates)

        clear_output(wait=True)
        print(f"Run: {run+1}/{max_runs}, Episode: {episode+1}/{max_episodes}: {step_rewards}")

    run_rewards.append(episode_rewards)
    updates_Intra.append(episode_updates)